# Classificação de Imagens de 03 classes de plantas por tipos de doenças, com Transfer Learning

#### Objetivo: Realizar o Teste de Inferência com imagens de testes de classe, para o cenário do Caso de Teste 04.

##### Classes Consideradas:
###### Grape___Black_rot
###### Grape___Esca_(Black_Measles)
###### Grape___healthy


## Consultas:
##### https://keras.io/applications/
##### https://www.tensorflow.org/tutorials/load_data/images
##### https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory
##### https://keras.io/api/data_loading/
##### https://datascience.stackexchange.com/questions/81541/how-to-predict-multiple-images-from-folder-in-python

## Leitura do local da base de dados

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
workdir_path = '/content/drive/MyDrive/TrabalhoFinal/DadosTCC/archive' #MODIFICAR!!! #Caminho para o seu workspace
os.chdir(workdir_path)

Mounted at /content/drive


In [2]:
os.path.abspath(os.getcwd())

'/content/drive/.shortcut-targets-by-id/1l60AC27n8nkWWiKY_Crqb8SeOohho5uQ/TrabalhoFinal/DadosTCC/archive'

In [3]:
# Base de imagens para treino com total de 4659 imagens (classes de tipos de platas)
trainpath = workdir_path + '/classeuva/Train'

# Base de imagens para validação com total 1169 imagens (classes de tipos de platas)
validpath = workdir_path + '/classeuva/Valid'

# Base de imagens para validação com total de 30 imagens (classes de tipos de platas)
testpath = workdir_path + '/classeuva/Test'


## Leitura de Bibliotecas

In [4]:
from tensorflow.keras.models import load_model
import argparse
import pickle
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.imagenet_utils import decode_predictions
import numpy as np
import logging, os
import sys 
from keras.preprocessing import image
import tensorflow as tf
import math
import operator
from pathlib import Path


## Leitura do modelo do respectivo cenário 06 para o Caso de Teste 04

In [5]:
# Ler modelo treinado gravado para submeter aos dados de testes
modelosalvo = tf.keras.models.load_model('/content/drive/MyDrive/TrabalhoFinal/DadosTCC/archive/Inferência-TestesModelos/Caso de Teste 04/VGG19-cenário06/Modelo_Salvo_1/VGG19.h5')

## Carrega as Imagens de Testes Selecionadas

In [6]:
# Prepara variáveis para imagens do tipo Healphy

Black_Measles_path = testpath +'/Black_Measles/' # Caminho das imagens

imagens_Black_Measles = [] # Inicializa uma lista para carregar as imagens


In [7]:
# Carrega as imagens para do tipo Grape___Esca_(Black_Measles)
for img in os.listdir(Black_Measles_path):                           # Cria um loop
        img = os.path.join(Black_Measles_path, img)                            
        img = tf.keras.utils.load_img(img, target_size=(224,224))    # coloca a imagem no shape 
        img = tf.keras.utils.img_to_array(img)                       # transforma em um array
        img = np.expand_dims(img, axis=0)                            
        imagem_normalizada = np.array(img, dtype="float") / 255.0    # normaliza a imagem dividindo por 255
        imagens_Black_Measles.append(imagem_normalizada)             # carrega a imagem para a lista
        
imagens_Black_Measles = np.vstack(imagens_Black_Measles)                         

## Previsão Dados de Teste (classe Black_Measles)

In [8]:
# faz a previsao das imagens de teste do tipo Black_Measles
dados_previsao_imagens_Black_Measles = []                           # inicializa uma lista para colocar o reultado da previsao das imagens na lista
nome_arquivos = []                                                  # inicializa uma lista para colocar o nome dos arquivos
for arquivo in os.listdir(Black_Measles_path):                      # inicializa um loop (para cada arquivo dentro da pasta)
    pred_result = modelosalvo.predict(imagens_Black_Measles)        # faz a previsao de cada arquivo
    dados_previsao_imagens_Black_Measles.append(pred_result)        # coloca os dados da previsão na lsita
    nome_arquivos.append(arquivo)                                   # coloca o nome do arquivo na lista "nome_arquivos"

1/1 [==============================] - 0s 37ms/step


In [9]:
dados_previsao_imagens_Black_Measles[0]

array([[1.7077365e-13, 1.0000000e+00, 1.9965475e-34],
       [3.7938962e-04, 9.9962056e-01, 7.7015854e-14],
       [9.3949948e-06, 9.9999058e-01, 1.1830265e-18],
       [7.9527184e-02, 9.2047209e-01, 8.2174546e-07],
       [2.1988787e-32, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [2.3211322e-03, 9.9767894e-01, 1.7599982e-11],
       [7.4365112e-04, 9.9925631e-01, 5.7913738e-13],
       [7.5490158e-07, 9.9999928e-01, 6.1884742e-22],
       [4.1147464e-23, 1.0000000e+00, 0.0000000e+00]], dtype=float32)

In [10]:
# Define os labels R:Grape___Black_rot, M:Grape___Esca_(Black_Measles) e H:Grape___healthy' na mesma ordem de classes do modelo na etapa de load.
# Previsão dados de testes selecionados

label_file_path = os.path.join("RMH", 'labels') 

#SORT
top_k = []
pred = [] 
for i in range(len(dados_previsao_imagens_Black_Measles)):
            ordem = dados_previsao_imagens_Black_Measles[i][0].argsort()[-len(dados_previsao_imagens_Black_Measles[i][0]):][::-1]
            top_k.append(ordem)
            top = top_k[i][:15]
            print(nome_arquivos[i])
            for node_id in top:
                planta_string = label_file_path[node_id]
                score = dados_previsao_imagens_Black_Measles[i][0][node_id]
                print('%s (score = %.5f)' % (planta_string, score))

8497f666-2c75-4e10-b7cc-fb1ca767b681___FAM_B.Msls 1279.JPG
M (score = 1.00000)
R (score = 0.00000)
H (score = 0.00000)
89265561-e2ab-4c9b-a649-909f2cbf5d01___FAM_B.Msls 4219.JPG
M (score = 1.00000)
R (score = 0.00000)
H (score = 0.00000)
8d9cecc8-1f75-4c7d-aa37-6c90882f4162___FAM_B.Msls 1222.JPG
M (score = 1.00000)
R (score = 0.00000)
H (score = 0.00000)
f8bff337-eefa-42cd-a196-b0b9832e6610___FAM_B.Msls 3908.JPG
M (score = 1.00000)
R (score = 0.00000)
H (score = 0.00000)
ffc0847a-e88d-4332-8c7c-4b63ba65a471___FAM_B.Msls 1130.JPG
M (score = 1.00000)
R (score = 0.00000)
H (score = 0.00000)
f83f01b3-32e6-4723-a440-bebfe2b56637___FAM_B.Msls 4038.JPG
M (score = 1.00000)
R (score = 0.00000)
H (score = 0.00000)
e4f51f0b-69a3-4446-bf0f-2aba1d283196___FAM_B.Msls 0868.JPG
M (score = 1.00000)
R (score = 0.00000)
H (score = 0.00000)
f4be6f93-2077-4fda-929b-10a90307e0c9___FAM_B.Msls 1266.JPG
M (score = 1.00000)
R (score = 0.00000)
H (score = 0.00000)
e7f1c789-431d-47bb-a223-fc58b3e734da___FAM_B.Msl